In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from skimage.measure import regionprops, label, find_contours
from scipy.spatial.distance import cdist

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255


In [ ]:
def load_Y_gray_with_gaussian_and_numeric(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []
    #image_files = os.listdir(folder_path)
    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(int(base))
        else:
            pass
        
    image_files.sort()
    #print(image_files)
    
    img = cv2.imread(folder_path + os.sep + str(image_files[0]).zfill(4) + ".png", cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + str(image_file).zfill(4) + ".png" ,
            cv2.IMREAD_GRAYSCALE
        )
        #print(image.shape)
        
        # ぼかし処理
        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
        
        
        
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

In [ ]:
def calc_iou(im1, im2):
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)
    iou_score = np.sum(intersection) / np.sum(union)
    if np.sum(union) == 0:
        iou_score = 0
    return iou_score

In [ ]:
baseline_imgs , _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/baseline/", normalize=False)
tap_imgs , _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/tap/", normalize=False)
suga_imgs , _ = load_Y_gray_with_gaussian_and_numeric("Z:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/tap_dataset/cropped_0_20/suga/", normalize=False)


In [ ]:
baseline_iou = list()

for i in range(baseline_imgs.shape[0] - 1):
    iou = calc_iou(baseline_imgs[i], baseline_imgs[i+1])
    baseline_iou.append(iou)
    
baseline_iou

In [ ]:
tap_iou = list()

for i in range(tap_imgs.shape[0] - 1):
    iou = calc_iou(tap_imgs[i], tap_imgs[i+1])
    tap_iou.append(iou)
    
tap_iou

In [ ]:
suga_iou = list()

for i in range(suga_imgs.shape[0] - 1):
    iou = calc_iou(suga_imgs[i], suga_imgs[i+1])
    suga_iou.append(iou)
    
suga_iou

In [ ]:
print("xy")
print("baseline : ", round(np.var(baseline_iou[4:14]), 4))
print("TAP : ", round(np.var(tap_iou[4:14]), 4))
print("Manual : ", round(np.var(suga_iou[2:14]), 4))